In [1]:
import os
import glob
import shutil

# 1. 경로 설정
BASE_GDRIVE_PATH = "/content/drive/MyDrive/DL_project"
ZIPS_PATH = os.path.join(BASE_GDRIVE_PATH, "zips")
TL_ZIP_FILE_PATH = os.path.join(ZIPS_PATH, "TL.zip")
LOCAL_LBL_PATH = "/content/train_labels_local"

print("🚀 TL.zip을 사용하여 라벨 데이터를 Colab 로컬 저장소로 빠르게 가져옵니다.")

# 2. Colab 로컬에 폴더 생성
os.makedirs(LOCAL_LBL_PATH, exist_ok=True)

# 3. TL.zip 파일 복사 및 압축 해제
print(f"\n[1/3] '{os.path.basename(TL_ZIP_FILE_PATH)}' 파일을 로컬로 복사합니다.")
shutil.copy(TL_ZIP_FILE_PATH, "/content/")

print("[2/3] 임시 폴더에 압축을 해제합니다.")
# -q (quiet) 옵션으로 조용히 실행, -d 옵션으로 대상 디렉터리 지정
!unzip -q /content/TL.zip -d /content/temp_unzip

# 4. 라벨 파일(.json)만 최종 로컬 경로로 복사
print("[3/3] 필요한 라벨 파일만 로컬 폴더로 복사합니다.")
label_source_path = '/content/temp_unzip/라벨/HW-OCR/4.Validation/P.Paper/R.Free'

# 'Argument list too long' 오류를 방지하기 위해 'find' 명령어 사용
!find {label_source_path} -name "*_4PR_*.json" -exec cp -t "{LOCAL_LBL_PATH}" {{}} +

# 5. 결과 확인 및 임시 파일 정리
copied_file_count = len(os.listdir(LOCAL_LBL_PATH))
print(f"\n✅ 복사 완료! {copied_file_count}개의 라벨 파일을 '{LOCAL_LBL_PATH}' 폴더로 가져왔습니다.")

print("\n🧹 임시 파일(TL.zip, temp_unzip 폴더)을 정리합니다.")
!rm /content/TL.zip
!rm -rf /content/temp_unzip
print("✅ 정리 완료!")

🚀 TL.zip을 사용하여 라벨 데이터를 Colab 로컬 저장소로 빠르게 가져옵니다.

[1/3] 'TL.zip' 파일을 로컬로 복사합니다.
[2/3] 임시 폴더에 압축을 해제합니다.
[3/3] 필요한 라벨 파일만 로컬 폴더로 복사합니다.

✅ 복사 완료! 31536개의 라벨 파일을 '/content/train_labels_local' 폴더로 가져왔습니다.

🧹 임시 파일(TL.zip, temp_unzip 폴더)을 정리합니다.
✅ 정리 완료!


In [5]:
import os
import json
from tqdm import tqdm
from pydantic import BaseModel, ValidationError
from typing import List

# ---------------------------------------------------------------------------
# 단어장 생성에 필요한 최소한의 Pydantic 모델 정의
# ---------------------------------------------------------------------------
class BBox(BaseModel):
    data: str

class LabelForVocab(BaseModel):
    bbox: List[BBox]

# ---------------------------------------------------------------------------
# 단어장 생성 및 저장 함수
# ---------------------------------------------------------------------------
def build_and_save_vocab(label_dir: str, output_file: str):
    """
    지정된 디렉토리의 모든 JSON 라벨 파일을 읽어 단어장을 만들고 파일로 저장합니다.
    """
    if not os.path.exists(label_dir):
        print(f"🚨 오류: '{label_dir}' 경로를 찾을 수 없습니다.")
        print("이전 단계의 데이터 준비 코드가 올바르게 실행되었는지 확인해주세요.")
        return

    print(f"'{label_dir}' 디렉토리에서 단어장 생성을 시작합니다...")

    char_set = set()
    all_label_files = os.listdir(label_dir)

    for label_file in tqdm(all_label_files, desc="라벨 파일 분석 중"):
        json_path = os.path.join(label_dir, label_file)

        # ✨✨✨ 오류 수정 부분 ✨✨✨
        # isdir()를 사용해 디렉토리인 경우 건너뛰거나, isfile() 또는 파일 확장자로 확인합니다.
        # 가장 간단한 방법은 파일 이름이 '.json'으로 끝나는지 확인하는 것입니다.
        if not label_file.endswith('.json'):
            continue
        # ✨✨✨✨✨✨✨✨✨✨✨✨✨

        try:
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                label = LabelForVocab(**data)
                for item in label.bbox:
                    char_set.update(item.data)
        except (json.JSONDecodeError, ValidationError, FileNotFoundError):
            # 오류가 있는 파일은 건너뜁니다.
            continue

    # 0번 인덱스에 패딩('<PAD>') 토큰을 추가하여 단어장을 만듭니다.
    char_list = sorted(list(char_set))
    char_list = ['<PAD>'] + char_list

    char_to_id = {char: i for i, char in enumerate(char_list)}
    id_to_char = {i: char for i, char in enumerate(char_list)}

    # 생성된 단어장을 파일로 저장
    vocab_data = {'char_to_id': char_to_id, 'id_to_char': id_to_char}
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(vocab_data, f, ensure_ascii=False, indent=4)

    print(f"\n✅ 단어장 생성 완료! 총 {len(char_to_id)}개의 고유 문자를 '{output_file}'에 저장했습니다.")

# ---------------------------------------------------------------------------
# 스크립트 실행 부분
# ---------------------------------------------------------------------------
if __name__ == '__main__':
    # 이전 단계에서 라벨 파일들을 복사해 둔 로컬 경로
    LOCAL_LBL_PATH = "/content/train_labels_local"

    # 생성된 vocab.json을 영구적으로 저장할 Google Drive 경로
    VOCAB_SAVE_PATH = "/content/drive/MyDrive/DL_project/vocab.json"

    # 실행!
    build_and_save_vocab(LOCAL_LBL_PATH, VOCAB_SAVE_PATH)

'/content/train_labels_local' 디렉토리에서 단어장 생성을 시작합니다...


라벨 파일 분석 중: 100%|██████████| 31536/31536 [00:07<00:00, 4109.69it/s]



✅ 단어장 생성 완료! 총 4109개의 고유 문자를 '/content/drive/MyDrive/DL_project/vocab.json'에 저장했습니다.
